# LangGraph practice#1 구성도 

![구성도](/home/ansgyqja/AI_application/images/1.practice.png)


In [5]:
import os,sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('utils'), '..')))
from module.utils import * 
from module.prompt import * 

## utils 에서 불러오기

In [6]:
# llm = get_gpt()
loader = get_pdf_loader()
text = get_text_splitter()
docs = get_docs(loader,text)
embedding = get_embedding()
retriever = get_retriever(docs=docs,embedding=embedding,k=5)
retriever_tool = get_retriever_tool(retriever=retriever)
tools = [retriever_tool]
tool_node = get_tool_node(retriever_tool)
# tavily = get_tavily_tool()
# tool = TavilySearch(max_results=3)
# tool_node = ToolNode(tools)
# tool_node = get_tool_node(tool)



### State 생성

In [7]:
from typing import TypedDict, Annotated, List, Literal
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    question : Annotated[list,add_messages] 
    answer : Annotated[str,"Answer"]
    is_relevant : Annotated[str,"yes or no"]
    messages : Annotated[list,add_messages]



In [11]:
query= '삼성전자 개발 AI'
llm = get_gemini()
llm_with_tools=llm.bind_tools(tools)
messages = llm_with_tools.invoke(query)
print(messages)

content='' additional_kwargs={'function_call': {'name': 'pdf_search', 'arguments': '{"query": "\\uc0bc\\uc131\\uc804\\uc790 \\uac1c\\ubc1c AI"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--1aa78b07-3d34-426a-87b4-b1fedc36d6fd-0' tool_calls=[{'name': 'pdf_search', 'args': {'query': '삼성전자 개발 AI'}, 'id': 'f9da38fa-121b-4caa-bbca-eb2918bcc3cc', 'type': 'tool_call'}] usage_metadata={'input_tokens': 52, 'output_tokens': 18, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}}


### Node 생성

In [ ]:
def agent(state:State)->State:
    llm = get_gemini()
    llm_with_tools=llm.bind_tools(tools)
    messages = llm_with_tools.invoke(state['question'][-1].content)
    return State({'messages':[messages]})

def tool_relevant(state : State):
    llm = get_gemini()
    is_relevant = get_relevant(llm)
    response = is_relevant.invoke(
        {"question": state["question"][-1].content, "context": state["messages"]}
    )
    return State({"is_relevant":response.score})

def is_retriever(state : State)-> Literal["generate","re_write_node"]:
    is_relevant = state.get("is_relevant","no")
    if is_relevant == "yes":
        return "generate"
    return "re_write_node"

def generate(state:State)->State:
    llm = get_gemini(temperature=0.5)
    prompt = get_rag_pt()
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({'question':state['question'][-1].content,'context':state['messages']})
    return State({'answer':answer})

# re write node
def re_write_node(state : State) -> State:
    chain = re_write_prompt | get_gemini(temperature=0.5) | StrOutputParser()
    response = chain.invoke({"question": state['question']})
    return State({"question":response})


### Graph 구성

In [ ]:
# from langgraph.prebuilt import tools_condition

state_graph = StateGraph(State)
state_graph.add_node('agent',agent)
state_graph.add_node('tools',tool_node)
state_graph.add_node('tool_relevant',tool_relevant)
state_graph.add_node('generate',generate)
state_graph.add_node('re_write_node',re_write_node)

state_graph.add_edge(START,'agent')
state_graph.add_conditional_edges(
    source='agent',
    path=tools_condition,
)
state_graph.add_edge('tools','tool_relevant')
state_graph.add_conditional_edges(
    source='tool_relevant',
    path=is_retriever,
)
state_graph.add_edge('re_write_node','agent')
state_graph.add_edge('generate',END)
state_graph.add_edge('agent',END)
graph = state_graph.compile(checkpointer=get_check_pointer())


In [ ]:
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

### 실행부

In [ ]:
config = get_runnable_config(thread_id=1)
# config 설정(재귀 최대 횟수, thread_id)

# question= '엔트로피 투자 금액'
question = '코스피 역대 최고점?'
# 질문 입력
inputs = State({'question':question})
# invoke_graph(graph, inputs, config)
stream_graph(graph, inputs, config)